# 🔇 Advanced Noise Cancellation & Safe Audio System

## Overview
This notebook demonstrates various noise cancellation techniques and implements a **Safe Audio Environment System** that keeps audio levels below **70 dB** for hearing protection.

### 🎯 Key Features:
- **Real-time noise reduction** using spectral subtraction
- **Adaptive volume control** to maintain safe dB levels
- **Environmental noise learning** for personalized filtering
- **Frequency analysis** and visualization
- **Hearing protection** with automatic limiting

### 📊 Safety Standard:
> **70 dB Threshold**: Sounds at or below 70 dB are generally considered safe for prolonged exposure without risk of hearing damage.

Let's explore different approaches to noise cancellation and audio safety!

In [2]:
# Install required packages for advanced noise cancellation
%pip install pyaudio numpy matplotlib scipy soundfile librosa pydub audioop-lts

import numpy as np
import matplotlib.pyplot as plt
import pyaudio
from scipy import signal
from scipy.fft import fft, ifft
import soundfile as sf
import librosa
from pydub import AudioSegment
import time
import threading
from collections import deque

print("✅ All packages installed and imported successfully!")
print("🔧 Setting up Safe Audio Environment System...")

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
✅ All packages installed and imported successfully!
🔧 Setting up Safe Audio Environment System...



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Dinesh\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
# 📊 Audio Safety: dB Level Calculation

def calculate_db_level(audio_data):
    """Calculate dB level of audio data"""
    if len(audio_data) == 0:
        return -np.inf
    
    # Calculate RMS (Root Mean Square)
    rms = np.sqrt(np.mean(audio_data ** 2))
    
    # Convert to dB relative to full scale
    if rms > 0:
        db_level = 20 * np.log10(rms)
        # Rough calibration to approximate real-world dB levels
        return db_level + 94  # Calibration offset
    return -np.inf

def is_safe_level(db_level, threshold=70.0):
    """Check if audio level is safe for prolonged exposure"""
    return db_level <= threshold

print("✅ Audio safety calculation functions loaded!")
print("📏 Functions: calculate_db_level(), is_safe_level()")

✅ Audio safety calculation functions loaded!
📏 Functions: calculate_db_level(), is_safe_level()


In [4]:
# 🛡️ Audio Safety: Limiting and Protection

def apply_safety_limiting(audio_data, max_db=70.0):
    """Apply safety limiting to keep audio below specified dB level"""
    current_db = calculate_db_level(audio_data)
    
    if current_db > max_db:
        # Calculate reduction factor needed
        reduction_db = current_db - max_db
        reduction_factor = 10 ** (-reduction_db / 20.0)
        
        # Apply smooth reduction
        audio_data = audio_data * reduction_factor
        
        # Apply soft limiting to prevent clipping
        audio_data = np.tanh(audio_data * 0.8) * 0.8
        
    return audio_data, calculate_db_level(audio_data)

print("✅ Safety limiting function loaded!")
print("🛡️ Function: apply_safety_limiting() - Keeps audio below 70dB")

✅ Safety limiting function loaded!
🛡️ Function: apply_safety_limiting() - Keeps audio below 70dB


In [5]:
# 🧪 Test Audio Safety Functions

print("🧪 Testing safety functions...")

# Generate test audio (1 second of noise)
test_audio = np.random.normal(0, 0.3, 44100)
original_db = calculate_db_level(test_audio)
safe_audio, safe_db = apply_safety_limiting(test_audio, max_db=70.0)

print(f"Original level: {original_db:.1f} dB")
print(f"After safety limiting: {safe_db:.1f} dB")
print(f"Safe for prolonged exposure: {is_safe_level(safe_db)}")

# Store results for visualization
print("📊 Test completed - Results ready for visualization")

🧪 Testing safety functions...
Original level: 83.5 dB
After safety limiting: 66.1 dB
Safe for prolonged exposure: True
📊 Test completed - Results ready for visualization


## 🎧 Device Selection & Connection Verification

In this section, we'll implement a robust system for:
- **Device Detection**: Finding available audio input/output devices
- **Connection Verification**: Testing actual hardware connection
- **User Interface**: Creating an intuitive selection system
- **Safety Checks**: Ensuring proper headphone connection before processing

The device selection system includes both automated detection and manual selection options.

In [6]:
# 🔍 Device Detection Functions

def get_audio_devices():
    """
    Get list of available audio input and output devices
    Returns: tuple of (input_devices, output_devices)
    """
    p = pyaudio.PyAudio()
    input_devices = []
    output_devices = []
    
    try:
        for i in range(p.get_device_count()):
            device_info = p.get_device_info_by_index(i)
            
            if device_info['maxInputChannels'] > 0:
                input_devices.append({
                    'index': i,
                    'name': device_info['name'],
                    'channels': device_info['maxInputChannels']
                })
            
            if device_info['maxOutputChannels'] > 0:
                output_devices.append({
                    'index': i,
                    'name': device_info['name'],
                    'channels': device_info['maxOutputChannels']
                })
    finally:
        p.terminate()
    
    return input_devices, output_devices

def test_device_connection(device_index, is_input=True, test_duration=0.1):
    """
    Test if a device is actually connected and functional
    Returns: True if device works, False otherwise
    """
    p = pyaudio.PyAudio()
    
    try:
        # Test parameters
        channels = 1
        rate = 44100
        chunk = 1024
        format = pyaudio.paFloat32
        
        if is_input:
            # Test input device
            stream = p.open(
                format=format,
                channels=channels,
                rate=rate,
                input=True,
                input_device_index=device_index,
                frames_per_buffer=chunk
            )
        else:
            # Test output device
            stream = p.open(
                format=format,
                channels=channels,
                rate=rate,
                output=True,
                output_device_index=device_index,
                frames_per_buffer=chunk
            )
        
        # Quick functional test
        if is_input:
            # Try to read some data
            stream.read(int(rate * test_duration))
        else:
            # Try to write some silence
            silence = np.zeros(int(rate * test_duration), dtype=np.float32)
            stream.write(silence.tobytes())
        
        stream.stop_stream()
        stream.close()
        return True
        
    except Exception as e:
        print(f"⚠️ Device test failed: {e}")
        return False
    finally:
        p.terminate()

print("✅ Device detection functions loaded")

✅ Device detection functions loaded


In [7]:
# 🎧 Headphone Detection & Verification

def find_headphone_devices():
    """
    Find devices that are likely to be headphones
    Returns: list of potential headphone devices
    """
    input_devices, output_devices = get_audio_devices()
    
    # Keywords that suggest headphones
    headphone_keywords = ['headphone', 'headset', 'earphone', 'buds', 'airpods', 'beats', 'sony', 'bose']
    
    potential_headphones = []
    
    for device in output_devices:
        device_name = device['name'].lower()
        
        # Check if device name contains headphone keywords
        for keyword in headphone_keywords:
            if keyword in device_name:
                # Test if device is actually connected
                if test_device_connection(device['index'], is_input=False):
                    potential_headphones.append(device)
                    break
    
    return potential_headphones

def verify_headphone_connection():
    """
    Comprehensive headphone verification
    Returns: True if headphones detected and connected, False otherwise
    """
    print("🔍 Scanning for headphone devices...")
    
    headphones = find_headphone_devices()
    
    if not headphones:
        print("❌ No headphone devices detected!")
        print("📱 Please ensure your headphones are:")
        print("   • Properly connected to your computer")
        print("   • Turned on (for wireless headphones)")
        print("   • Set as the default audio device")
        return False
    
    print(f"✅ Found {len(headphones)} headphone device(s):")
    for i, device in enumerate(headphones, 1):
        print(f"   {i}. {device['name']} (Device {device['index']})")
    
    return True

print("✅ Headphone detection functions loaded")

✅ Headphone detection functions loaded


In [8]:
# 🖥️ Device Selection GUI

def create_device_selection_gui():
    """
    Create an interactive GUI for device selection
    Returns: tuple of (selected_input_index, selected_output_index)
    """
    import tkinter as tk
    from tkinter import ttk, messagebox
    
    # Get available devices
    input_devices, output_devices = get_audio_devices()
    
    if not output_devices:
        messagebox.showerror("Error", "No output devices found!")
        return None, None
    
    # Create GUI window
    root = tk.Tk()
    root.title("🎧 Audio Device Selection")
    root.geometry("500x400")
    
    # Variables to store selections
    selected_input = tk.IntVar(value=0 if input_devices else -1)
    selected_output = tk.IntVar(value=0)
    result = {'input': None, 'output': None, 'confirmed': False}
    
    # Title
    title_label = tk.Label(root, text="Select Audio Devices", font=("Arial", 16, "bold"))
    title_label.pack(pady=10)
    
    # Input devices section
    if input_devices:
        input_frame = tk.LabelFrame(root, text="🎤 Input Devices (Microphone)", font=("Arial", 12))
        input_frame.pack(fill="x", padx=20, pady=10)
        
        for i, device in enumerate(input_devices):
            status = "✅ Connected" if test_device_connection(device['index'], is_input=True) else "❌ Not Connected"
            tk.Radiobutton(
                input_frame,
                text=f"{device['name']} - {status}",
                variable=selected_input,
                value=i,
                font=("Arial", 10)
            ).pack(anchor="w", padx=10, pady=2)
    
    # Output devices section
    output_frame = tk.LabelFrame(root, text="🎧 Output Devices (Headphones/Speakers)", font=("Arial", 12))
    output_frame.pack(fill="x", padx=20, pady=10)
    
    for i, device in enumerate(output_devices):
        status = "✅ Connected" if test_device_connection(device['index'], is_input=False) else "❌ Not Connected"
        tk.Radiobutton(
            output_frame,
            text=f"{device['name']} - {status}",
            variable=selected_output,
            value=i,
            font=("Arial", 10)
        ).pack(anchor="w", padx=10, pady=2)
    
    # Confirm button
    def confirm_selection():
        if input_devices and selected_input.get() >= 0:
            result['input'] = input_devices[selected_input.get()]['index']
        
        if selected_output.get() >= 0:
            result['output'] = output_devices[selected_output.get()]['index']
        
        result['confirmed'] = True
        root.destroy()
    
    confirm_btn = tk.Button(
        root,
        text="✅ Confirm Selection",
        command=confirm_selection,
        font=("Arial", 12),
        bg="#4CAF50",
        fg="white",
        pady=5
    )
    confirm_btn.pack(pady=20)
    
    # Warning label
    warning_label = tk.Label(
        root,
        text="⚠️ Please ensure headphones are connected before proceeding",
        font=("Arial", 10),
        fg="red"
    )
    warning_label.pack(pady=5)
    
    root.mainloop()
    
    if result['confirmed']:
        return result['input'], result['output']
    else:
        return None, None

print("✅ Device selection GUI functions loaded")

✅ Device selection GUI functions loaded


## 🎵 Real-Time Audio Processing & Noise Cancellation

Now we'll implement the core audio processing system that includes:
- **Real-time Audio Capture**: Continuous input from microphone
- **Noise Cancellation**: Advanced filtering techniques
- **Safety Monitoring**: Continuous volume level checking
- **Live Output**: Processed audio to headphones

The system processes audio in chunks for real-time performance while maintaining audio quality.

In [9]:
# 🎛️ Noise Cancellation Algorithms

def spectral_subtraction(audio_data, noise_factor=2.0):
    """
    Spectral subtraction noise reduction
    Reduces noise by subtracting estimated noise spectrum
    """
    # Convert to frequency domain
    fft_data = np.fft.fft(audio_data)
    
    # Estimate noise (first 10% of signal)
    noise_length = len(audio_data) // 10
    noise_spectrum = np.mean(np.abs(np.fft.fft(audio_data[:noise_length])))
    
    # Apply spectral subtraction
    magnitude = np.abs(fft_data)
    phase = np.angle(fft_data)
    
    # Subtract noise estimate
    clean_magnitude = magnitude - noise_factor * noise_spectrum
    clean_magnitude = np.maximum(clean_magnitude, 0.1 * magnitude)  # Floor to prevent artifacts
    
    # Reconstruct signal
    clean_fft = clean_magnitude * np.exp(1j * phase)
    clean_audio = np.real(np.fft.ifft(clean_fft))
    
    return clean_audio.astype(np.float32)

def wiener_filter(audio_data, noise_power_ratio=0.1):
    """
    Wiener filter for noise reduction
    Optimal filter that minimizes mean square error
    """
    # Convert to frequency domain
    fft_data = np.fft.fft(audio_data)
    power_spectrum = np.abs(fft_data) ** 2
    
    # Estimate signal and noise power
    signal_power = np.mean(power_spectrum)
    noise_power = signal_power * noise_power_ratio
    
    # Wiener filter transfer function
    wiener_filter_h = signal_power / (signal_power + noise_power)
    
    # Apply filter
    filtered_fft = fft_data * wiener_filter_h
    filtered_audio = np.real(np.fft.ifft(filtered_fft))
    
    return filtered_audio.astype(np.float32)

def bandpass_filter(audio_data, low_freq=300, high_freq=3400, sample_rate=44100):
    """
    Bandpass filter to remove frequencies outside speech range
    Effective for voice communication
    """
    from scipy import signal
    
    # Design bandpass filter
    nyquist = sample_rate * 0.5
    low = low_freq / nyquist
    high = high_freq / nyquist
    
    b, a = signal.butter(4, [low, high], btype='band')
    
    # Apply filter
    filtered_audio = signal.filtfilt(b, a, audio_data)
    
    return filtered_audio.astype(np.float32)

print("✅ Noise cancellation algorithms loaded")

✅ Noise cancellation algorithms loaded


In [10]:
# 🔄 Real-Time Audio Stream Processing

def create_audio_streams(input_device_index, output_device_index, chunk_size=1024, sample_rate=44100):
    """
    Create input and output audio streams
    Returns: tuple of (input_stream, output_stream, audio_interface)
    """
    p = pyaudio.PyAudio()
    
    try:
        # Create input stream
        input_stream = p.open(
            format=pyaudio.paFloat32,
            channels=1,
            rate=sample_rate,
            input=True,
            input_device_index=input_device_index,
            frames_per_buffer=chunk_size
        )
        
        # Create output stream
        output_stream = p.open(
            format=pyaudio.paFloat32,
            channels=1,
            rate=sample_rate,
            output=True,
            output_device_index=output_device_index,
            frames_per_buffer=chunk_size
        )
        
        return input_stream, output_stream, p
        
    except Exception as e:
        p.terminate()
        raise Exception(f"Failed to create audio streams: {e}")

def process_audio_chunk(audio_chunk, noise_reduction_method='spectral'):
    """
    Process a single chunk of audio with noise reduction
    Returns: processed audio chunk
    """
    # Convert bytes to numpy array
    audio_data = np.frombuffer(audio_chunk, dtype=np.float32)
    
    if len(audio_data) == 0:
        return audio_chunk
    
    # Apply noise reduction based on selected method
    if noise_reduction_method == 'spectral':
        processed_audio = spectral_subtraction(audio_data)
    elif noise_reduction_method == 'wiener':
        processed_audio = wiener_filter(audio_data)
    elif noise_reduction_method == 'bandpass':
        processed_audio = bandpass_filter(audio_data)
    else:
        processed_audio = audio_data  # No processing
    
    # Apply safety limiting
    processed_audio, _ = apply_safety_limiting(processed_audio)
    
    return processed_audio.tobytes()

def start_noise_cancellation(input_device, output_device, duration_seconds=30, method='spectral'):
    """
    Start real-time noise cancellation processing
    """
    print(f"🎧 Starting noise cancellation for {duration_seconds} seconds...")
    print(f"🎛️ Using {method} noise reduction")
    print("🔊 Processing audio in real-time...")
    
    # Verify headphone connection one more time
    if not verify_headphone_connection():
        print("❌ Cannot start - headphones not properly connected!")
        return False
    
    try:
        # Create audio streams
        input_stream, output_stream, p = create_audio_streams(input_device, output_device)
        
        # Process audio for specified duration
        chunks_to_process = int(duration_seconds * 44100 / 1024)
        
        for i in range(chunks_to_process):
            # Read audio chunk
            audio_chunk = input_stream.read(1024, exception_on_overflow=False)
            
            # Process the chunk
            processed_chunk = process_audio_chunk(audio_chunk, method)
            
            # Output processed audio
            output_stream.write(processed_chunk)
            
            # Progress indicator
            if i % 43 == 0:  # Roughly every second
                print(f"⏱️ Processing... {i//43 + 1}s")
        
        # Cleanup
        input_stream.stop_stream()
        output_stream.stop_stream()
        input_stream.close()
        output_stream.close()
        p.terminate()
        
        print("✅ Noise cancellation completed successfully!")
        return True
        
    except Exception as e:
        print(f"❌ Error during processing: {e}")
        return False

print("✅ Real-time processing functions loaded")

✅ Real-time processing functions loaded


## 🚀 Live Demonstration & Usage

This section provides a complete demonstration of the noise cancellation system:
- **Device Setup**: Automated device detection and verification
- **Safety Check**: Ensuring proper headphone connection
- **Live Processing**: Real-time noise cancellation with multiple algorithms
- **Performance Monitoring**: Audio level tracking and safety compliance

Run the cells below to experience the complete noise cancellation system in action!

In [11]:
# 🎯 Complete Noise Cancellation Demo

def run_complete_demo():
    """
    Run the complete noise cancellation demonstration
    """
    print("🎧 ADVANCED NOISE CANCELLATION SYSTEM")
    print("=" * 50)
    
    # Step 1: Device Detection
    print("\n📱 Step 1: Device Detection")
    input_devices, output_devices = get_audio_devices()
    
    print(f"Found {len(input_devices)} input device(s)")
    print(f"Found {len(output_devices)} output device(s)")
    
    # Step 2: Headphone Verification
    print("\n🎧 Step 2: Headphone Verification")
    if not verify_headphone_connection():
        print("❌ Demo cannot continue without proper headphones!")
        return
    
    # Step 3: Device Selection
    print("\n🖥️ Step 3: Device Selection")
    print("Opening device selection GUI...")
    
    input_device, output_device = create_device_selection_gui()
    
    if input_device is None or output_device is None:
        print("❌ No devices selected. Demo cancelled.")
        return
    
    print(f"✅ Selected input device: {input_device}")
    print(f"✅ Selected output device: {output_device}")
    
    # Step 4: Choose processing method
    print("\n🎛️ Step 4: Noise Reduction Method")
    print("Available methods:")
    print("1. Spectral Subtraction (recommended)")
    print("2. Wiener Filter")
    print("3. Bandpass Filter")
    
    methods = ['spectral', 'wiener', 'bandpass']
    method = methods[0]  # Default to spectral
    
    print(f"Using: {method} method")
    
    # Step 5: Start Processing
    print("\n🚀 Step 5: Live Processing")
    print("⚠️ SAFETY NOTICE: Audio levels are automatically limited to 70dB")
    
    success = start_noise_cancellation(
        input_device=input_device,
        output_device=output_device,
        duration_seconds=10,  # Shorter duration for demo
        method=method
    )
    
    if success:
        print("\n🎉 Demo completed successfully!")
        print("🔊 Your audio was processed with noise cancellation")
        print("🛡️ All safety limits were maintained")
    else:
        print("\n❌ Demo encountered an error")
    
    print("\n📊 Demo Summary:")
    print(f"• Device verification: ✅")
    print(f"• Safety monitoring: ✅")
    print(f"• Noise reduction: ✅")
    print(f"• Real-time processing: ✅")

# Quick Test Function
def quick_safety_test():
    """
    Quick test of safety functions without full processing
    """
    print("🧪 Quick Safety Test")
    print("-" * 30)
    
    # Test audio safety functions
    test_audio = np.random.normal(0, 0.5, 44100)  # 1 second of test audio
    
    original_db = calculate_db_level(test_audio)
    safe_audio, safe_db = apply_safety_limiting(test_audio, max_db=70.0)
    
    print(f"📊 Original level: {original_db:.1f} dB")
    print(f"📊 After limiting: {safe_db:.1f} dB")
    print(f"🛡️ Safe for prolonged exposure: {is_safe_level(safe_db)}")
    
    if verify_headphone_connection():
        print("🎧 Headphones: Connected and verified")
    else:
        print("⚠️ Headphones: Not detected or not connected")

print("✅ Demo functions ready!")
print("\n🎯 Available commands:")
print("• run_complete_demo() - Full noise cancellation demonstration")
print("• quick_safety_test() - Test safety systems only")
print("• verify_headphone_connection() - Check headphone status")

✅ Demo functions ready!

🎯 Available commands:
• run_complete_demo() - Full noise cancellation demonstration
• quick_safety_test() - Test safety systems only
• verify_headphone_connection() - Check headphone status


In [12]:
# 🎬 Execute Demonstration

# Uncomment the line below to run the complete demonstration
# run_complete_demo()

# Or run a quick safety test first
quick_safety_test()

print("\n" + "="*60)
print("🎧 NOISE CANCELLATION SYSTEM READY")
print("="*60)
print("📋 To start the full demonstration:")
print("   run_complete_demo()")
print("\n📋 To test individual components:")
print("   • quick_safety_test() - Test safety systems")
print("   • verify_headphone_connection() - Check headphones")
print("   • get_audio_devices() - List available devices")
print("\n⚠️ IMPORTANT SAFETY NOTES:")
print("   • Always wear headphones when running demonstrations")
print("   • Audio levels are automatically limited to 70dB")
print("   • Stop immediately if you experience discomfort")
print("="*60)

🧪 Quick Safety Test
------------------------------
📊 Original level: 88.0 dB
📊 After limiting: 66.1 dB
🛡️ Safe for prolonged exposure: True
🔍 Scanning for headphone devices...
⚠️ Device test failed: [Errno -9999] Unanticipated host error
❌ No headphone devices detected!
📱 Please ensure your headphones are:
   • Properly connected to your computer
   • Turned on (for wireless headphones)
   • Set as the default audio device
⚠️ Headphones: Not detected or not connected

🎧 NOISE CANCELLATION SYSTEM READY
📋 To start the full demonstration:
   run_complete_demo()

📋 To test individual components:
   • quick_safety_test() - Test safety systems
   • verify_headphone_connection() - Check headphones
   • get_audio_devices() - List available devices

⚠️ IMPORTANT SAFETY NOTES:
   • Always wear headphones when running demonstrations
   • Audio levels are automatically limited to 70dB
   • Stop immediately if you experience discomfort
⚠️ Device test failed: [Errno -9999] Unanticipated host error
❌